# Simple Cryptocurrency Proof of Concept in Ruby

In [1]:
require 'openssl'
require 'minitest'
require 'benchmark'
require 'base64'

# Enable assertions outside of Minitest::Test 
include Minitest::Assertions
Object.class_eval do
  attr_accessor :assertions
end
self.assertions = 0

# Override IRuby::Kernal to hide backtrace from display
# https://github.com/SciRuby/iruby/blob/master/lib/iruby/kernel.rb
IRuby::Kernel.class_eval do
  def error_message(e)
    { status: :error,
        ename: e.class.to_s,
        evalue: e.message,
        traceback: ["\e[31m#{e.class}\e[0m: #{e.message}"],
        execution_count: @execution_count }
    end
end


:error_message

## The Miner

The miner takes in a message & finds a `nonce` that satisfies a POW difficulty level

In [2]:
module Miner
  def mine(message, difficulty_level = 2) # 2 leading zeros
    nonce = 0
    loop do
      hash = OpenSSL::Digest::SHA256.new(message + nonce.to_s).to_s
      return [hash, nonce] if hash.start_with? '0' * difficulty_level
      nonce += 1
    end
  end
end

:mine

Time required to mine increases exponentially when difficulty increases. Difficulty would increase according to the moving average of time for each block to be added to the blockchain.

In [3]:
class TestMiner; include Miner; end

puts Benchmark.measure { puts TestMiner.new.mine 'bar' }
puts Benchmark.measure { puts TestMiner.new.mine 'bar', 4 }
# puts Benchmark.measure { puts TestMiner.new.mine 'bar', 6 }

["00b894f575e9311a064511b37bc5bfd57365980ba9157aaf69afec3fecf8178a", 5]
  0.000000   0.000000   0.000000 (  0.000196)

["000065a302ba3d5ff98dd372b82ab1837fbbbae10c4576a9b17d778dfda89955", 61125]
  0.210000   0.000000   0.210000 (  0.217808)



## Wallets

Here we're creating a simple coin wallet with a set of Public/Private keys using a 512 bit RSA for Public Key encryption. 

We'll simply use the an URL safe base64 encoded Public Key as the wallet address.

In [4]:
class Wallet
  attr_reader :address
  
  def initialize
    @key = OpenSSL::PKey::RSA.new(512)  # obviously insecure
    @address = Base64.urlsafe_encode64(@key.public_key.to_der)
  end
  
  def sign(message)
    digest = OpenSSL::Digest::SHA256.new(message)
    sig = @key.sign(digest, message)
    Base64.urlsafe_encode64(sig)
  end
end

w = Wallet.new


#<Wallet:0x007ff9a7509648 @key=#<OpenSSL::PKey::RSA:0x007ff9a75095d0>, @address="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANv16ux--mK-kw81A2bFG0AOngSW7JESdgxleUICqD2XMw9-G5zY_6xtpoen8CPwafVuUQgbM7FjkC4unEgYhI8CAwEAAQ==">

And then we want to have some helper methods to verify the encryption.

In [5]:
module PKeyVerifier
  def verify(pkey_base64, signature_base64, message)
    pkey = Base64.urlsafe_decode64(pkey_base64)
    signature = Base64.urlsafe_decode64(signature_base64)
    
    key =  OpenSSL::PKey::RSA.new(pkey)
    digest = OpenSSL::Digest::SHA256.new(message)
    key.verify(digest, signature, message)
  end
end

class TestVerifier; include PKeyVerifier; end
pkey_verifier = TestVerifier.new

#<TestVerifier:0x007ff9a74ebc88>

In [6]:
signature = w.sign('some transaction')
assert pkey_verifier.verify(w.address, signature, 'some transaction')

true

In [7]:
assert pkey_verifier.verify(w.address, signature, 'some altered transaction'), 'Message altered'

Minitest::Assertion: Message altered

In [8]:
another_wallet = Wallet.new
assert pkey_verifier.verify(another_wallet.address, signature, 'some transaction'), 'Message not signed by same wallet'

Minitest::Assertion: Message not signed by same wallet

## Transactions

A transaction would need to have a `source` wallet, and an instructions of amounts to send to a `destination` wallets. 

In [9]:
class Transaction
  attr_reader :source_wallet, :instruction
  
  def initialize(source_wallet, instructions)
    @source_wallet = source_wallet
    @instructions = instructions
  end
end

:initialize

## Transaction Instructions

We'll define a class for Transaction Instructions. Each transaction instruction will define the target wallet & the amount to send.

In [10]:
class Transaction::Instruction
  attr_reader :target_wallet, :amount
  
  def initialize(target_wallet, amount)
    @target_wallet = target_wallet
    @amount = amount.to_f
  end
  
  def to_hash
    { 
      target:  @target_wallet,
      amount: @amount
    }
  end
end

:to_hash

So a typical transaction will look more of less like so 

In [11]:
w1 = Wallet.new
w2 = Wallet.new

t1 = Transaction.new(w1, Transaction::Instruction.new(w2, 25.0))

nil  # hide noisy output

## Transactions List

In a ledger system, all transactions will be a chronologically ordered list. In our case, we'll just add them into a global variable as soon as each transaction is defined.

In [12]:
class Transaction
  attr_reader :source_wallet, :instruction
  
  def initialize(source_wallet, instruction)
    @source_wallet = source_wallet
    @instruction = instruction
    
    append_to_transactions_list
  end
  
  def to_hash
    {
      source: @source_wallet,
      instruction: @instruction.to_hash
    }
  end
  
  def append_to_transactions_list
    $transaction_list << self
  end
end


:append_to_transactions_list

Now, according to the Bitcoin whitepaper, each transaction owner should digitally sign a hash of previous transaction with public key of next owner, so that a payee can verify the chain of ownership.

To achieve that, we'll need to find the last transaction of the `source` wallet, and use it as create a hash & signature. Time to tweak `Transaction`.

In [13]:
class Transaction
  attr_reader :hash, :signature, :previous_hash
  
  def initialize(source_wallet, instruction, hash, signature, previous_hash)
    @source_wallet = source_wallet
    @instruction = instruction
    @hash = hash
    @signature = signature
    @previous_hash = previous_hash
    
    append_to_transactions_list
  end

  def to_hash 
    {
      owner: @source_wallet,
      instruction: @instruction.to_hash,
      hash: @hash,
      signature: @signature,
      previous_hash: @previous_hash,
    }
  end
  
end

:to_hash

In [14]:
class Wallet
  def sign_instruction(instruction)
    previous_hash = last_wallet_transaction && last_wallet_transaction.hash
    hash_payload  = {
      target_address: instruction.target_wallet,
      previous_hash: previous_hash,
      instruction: instruction.to_hash
    }
    hash = OpenSSL::Digest::SHA256.new(hash_payload.to_s)
    signature = sign(hash.to_s)
    
    [hash, signature, previous_hash]
  end
  
  def last_wallet_transaction
    $transaction_list.select {|t| t.source_wallet == self.address}.last
  end
end


:last_wallet_transaction

In [15]:
$transaction_list = []

w1 = Wallet.new
w2 = Wallet.new
w3 = Wallet.new


# We'll pretend that $transaction_list is being transferred through the network as the public ledger
# Since Transactions will be public, all data stored within should be kosher for public viewing (i.e. no private keys)
t1 = Transaction.new(w1.address, i = Transaction::Instruction.new(w2.address, 25.0), *w1.sign_instruction(i))
t2 = Transaction.new(w1.address, i = Transaction::Instruction.new(w2.address, 10.0), *w1.sign_instruction(i))
t3 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 5.0), *w2.sign_instruction(i))
t4 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 2.0), *w2.sign_instruction(i))

nil  # hide noisy output

In [16]:
def inspect_all_transactions
  $transaction_list.map(&:to_hash).each_with_index do |t, index|
    source_address = t[:owner][33..42]  # for display purposes
    target_address = t[:instruction][:target][33..42]  # for display purposes

    puts "Transaction ##{index+1}: \n"
    puts "From: \t\t\t#{source_address}"
    puts "To: \t\t\t\t#{target_address}"
    puts "Amount: \t\t#{t[:instruction][:amount]}"
    puts "Hash: \t\t\t#{t[:hash]}"
    puts "Signature: \t\t#{t[:signature]}"
    puts "Previous Hash: \t#{t[:previous_hash]}"
    puts "----------------------\n"
  end
end

inspect_all_transactions

nil  # hide noisy output

Transaction #1: 

From: 			LHK-Cutb_1
To: 				LAsia2fh1v
Amount: 		25.0
Hash: 			8ed94054ba290dcc56ddb46aa127a7f7574f7d0a38d9ac61bb63d5f18cd5a369
Signature: 		CqBK4Al6rs1UT7DEG5h6ieNiYBCsXAFv7pnp0oNt74xeG1AeI2aTK47nK-BSQJHgg_-oxNIVH_yYEy2ZDj98Lw==
Previous Hash: 	
----------------------

Transaction #2: 

From: 			LHK-Cutb_1
To: 				LAsia2fh1v
Amount: 		10.0
Hash: 			6565233d6ff6c789746c1124396721a8fffcf10b25cfd7c9dd8a13c6cdf3565b
Signature: 		lNgL14_g5vH6CaXb6SOWVs2-UPShTwsn4fYLP_zS39kJ4kzw4Sq_4QcqovTCOQb_5bOWCHRCcZ6n9cxfogZ1Qg==
Previous Hash: 	8ed94054ba290dcc56ddb46aa127a7f7574f7d0a38d9ac61bb63d5f18cd5a369
----------------------

Transaction #3: 

From: 			LAsia2fh1v
To: 				JWysX4287s
Amount: 		5.0
Hash: 			b018485c585c0b4954f2ff4f0b25fec2a60a69cd10d0d5c8f458c2882d46eaf5
Signature: 		QJSOTs59lvSJZOG7N8YdEJtygOiJXNlUUTow9FpUTt3B5figvNrAYNZkl2j2geWnhQnucCd1lKuD7Kb2UnteXg==
Previous Hash: 	
----------------------

Transaction #4: 

From: 			LAsia2fh1v
To: 				JWysX4287s
Amount: 		2.0

So as the payees, the data integrity can be verified through the signature chains.

In [17]:
class Transaction
  include PKeyVerifier

  def self.find_transaction(hash)
    $transaction_list.find { |t| t.hash == hash }
  end
  
  def verify_self_and_ancestors
    verify_transaction
    
    return true unless @previous_hash
    self.class.find_transaction(@previous_hash).verify_self_and_ancestors
  end
  
  def verify_transaction
    raise 'Invalid transaction' unless verify(@source_wallet, @signature, recalculate_hash.to_s)
    true
  end
  
  def recalculate_hash
    hash_payload = {
      target_address: @instruction.target_wallet,
      previous_hash: previous_transaction && previous_transaction.hash,
      instruction: @instruction.to_hash
    }
    OpenSSL::Digest::SHA256.new(hash_payload.to_s)
  end
  
  def previous_transaction
    self.class.find_transaction(@previous_hash)
  end
end

t1.verify_self_and_ancestors

true

Changes to the transaction instructions or target wallets are quickly detectable throughout the chain.

In [18]:
hacker_wallet = Wallet.new

t2.instance_variable_set(:@instruction, Transaction::Instruction.new(hacker_wallet.address, 10.0))
t2.verify_self_and_ancestors

RuntimeError: Invalid transaction

## Tampering with the public ledger

### Reversing transactions

It is still possible to alter the transactions within the ledger if those Transactions originated from the attacker's wallet, since they can re-sign the hashes, allowing them to reverse any transactions made by them.

In [19]:
$transaction_list = []

t1 = Transaction.new(hacker_wallet.address, i = Transaction::Instruction.new(w2.address, 25.0), *hacker_wallet.sign_instruction(i))
t2 = Transaction.new(hacker_wallet.address, i = Transaction::Instruction.new(w2.address, 10.0), *hacker_wallet.sign_instruction(i))
t3 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 5.0), *w2.sign_instruction(i))
t4 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 2.0), *w2.sign_instruction(i))

inspect_all_transactions

nil  # hide noisy output

Transaction #1: 

From: 			JrsQeGd9kt
To: 				LAsia2fh1v
Amount: 		25.0
Hash: 			8ed94054ba290dcc56ddb46aa127a7f7574f7d0a38d9ac61bb63d5f18cd5a369
Signature: 		eOD6CqmQOY-uWLetgIfm0UIowndqqFexMQlMndWxDMTMWtKXKcfuCO31y9qdMywuU3QBid834RSH6DjuIrHg0A==
Previous Hash: 	
----------------------

Transaction #2: 

From: 			JrsQeGd9kt
To: 				LAsia2fh1v
Amount: 		10.0
Hash: 			6565233d6ff6c789746c1124396721a8fffcf10b25cfd7c9dd8a13c6cdf3565b
Signature: 		VrVGjPSb8pt9KlF6RreyAWrIyo06lx-TU30aIXcfr5z2eGR0m3u7cnwmSz_LNcPah5HVERpuqeq0jg7JooEDaA==
Previous Hash: 	8ed94054ba290dcc56ddb46aa127a7f7574f7d0a38d9ac61bb63d5f18cd5a369
----------------------

Transaction #3: 

From: 			LAsia2fh1v
To: 				JWysX4287s
Amount: 		5.0
Hash: 			b018485c585c0b4954f2ff4f0b25fec2a60a69cd10d0d5c8f458c2882d46eaf5
Signature: 		QJSOTs59lvSJZOG7N8YdEJtygOiJXNlUUTow9FpUTt3B5figvNrAYNZkl2j2geWnhQnucCd1lKuD7Kb2UnteXg==
Previous Hash: 	
----------------------

Transaction #4: 

From: 			LAsia2fh1v
To: 				JWysX4287s
Amount: 		2.0

In [20]:
def hacker_wallet.l33t_sign_instruction(instruction, current_transaction)
  previous_hash = current_transaction.previous_hash
  hash_payload  = {
    target_address: instruction.target_wallet,
    previous_hash: previous_hash,
    instruction: instruction.to_hash
  }
  hash = OpenSSL::Digest::SHA256.new(hash_payload.to_s)
  signature = sign(hash.to_s)

  [hash, signature, previous_hash]
end

# Reversing 25.0 transfer to w2
i = Transaction::Instruction.new(t1.instruction.target_wallet, 0.0)
hash, signature, previous_hash = hacker_wallet.l33t_sign_instruction(i, t1)
t1.instance_variable_set(:@instruction, i)
t1.instance_variable_set(:@hash, hash)
t1.instance_variable_set(:@signature, signature)

# Update t2's previous_hash
t2.instance_variable_set(:@previous_hash, hash)

# Recalculate hash for all later transactions
i = Transaction::Instruction.new(t2.instruction.target_wallet, t2.instruction.amount)
hash, signature, previous_hash = hacker_wallet.l33t_sign_instruction(i, t2)
t2.instance_variable_set(:@hash, hash)
t2.instance_variable_set(:@signature, signature)


# Passes tests
assert t1.verify_self_and_ancestors
assert t2.verify_self_and_ancestors

true

In [21]:
inspect_all_transactions

nil  # hide noisy output

Transaction #1: 

From: 			JrsQeGd9kt
To: 				LAsia2fh1v
Amount: 		0.0
Hash: 			533491acf7ec311bf60659c63fa11a8bcc1b346dbfffa44f9038da9d36c539b8
Signature: 		hTMlMANDw5YoT9IkrCPrI2cA6E60Ku13TYK9Yf7wJ2ZgWuYSIYgDcU5IZ8VSgXT8Ah43C03B9eJFjjhColR-yA==
Previous Hash: 	
----------------------

Transaction #2: 

From: 			JrsQeGd9kt
To: 				LAsia2fh1v
Amount: 		10.0
Hash: 			b24cc3ee700c5a7c8c70fe3aac1553df930b5052c5405a82ebe1e7796398a8c6
Signature: 		IAHh_uTe5MRiQvM0kActulOKN6BUl7XOzKF947v3IV2T45afL3quQtgNWg77feH2VsWyqAGCj9eqUxxbWLG5jw==
Previous Hash: 	533491acf7ec311bf60659c63fa11a8bcc1b346dbfffa44f9038da9d36c539b8
----------------------

Transaction #3: 

From: 			LAsia2fh1v
To: 				JWysX4287s
Amount: 		5.0
Hash: 			b018485c585c0b4954f2ff4f0b25fec2a60a69cd10d0d5c8f458c2882d46eaf5
Signature: 		QJSOTs59lvSJZOG7N8YdEJtygOiJXNlUUTow9FpUTt3B5figvNrAYNZkl2j2geWnhQnucCd1lKuD7Kb2UnteXg==
Previous Hash: 	
----------------------

Transaction #4: 

From: 			LAsia2fh1v
To: 				JWysX4287s
Amount: 		2.0


### Double Spending

To be continued ...

## Why not generate a hash chain using the last transaction?

Need more research on this. Let me know if you know the answer :D

This will just ensure a totally chronological chain of transactions, and the reversal can only be done before another wallet transaction is added to the end. So why not use one single transaction chain?

Maybe, since lots of transactions could be created at any given time, making the last transaction hard to determine. Also it would be too slow for a consensus to made by the entire network to determine which transaction is the last transaction due to the sheer number of transactions made by each node at any given time. 

Maybe that's why transactions are verified by Blocks instead?

To be continued ...

In [ ]:
#